## **K=100**

In [1]:
import random
import math

def media_recursiva(media_anterior, valor_actual, j):
    x=(media_anterior + ( (valor_actual - media_anterior) / (j + 1.0)))
    return x

def varianza_recursiva(media_anterior, media_actual, varianza_anterior, j):
    p1 = (1 - ( 1.0 / j)) * varianza_anterior
    p2 = ((media_actual - media_anterior) ** 2 ) * (j + 1)
    s=(p1 + p2)
    return s

def get_random_number():
    """
    Genera una variable aleatoria
    con una distribución uniforme.
    """
    return random.random()

def get_exponential(lambda_parameter):
    """
    Genera una variable aleatoria
    con una distribución exponencial.
    """
    random_number = get_random_number()
    value = - (math.log(1 - random_number) / float(lambda_parameter))
    return value

def get_normal():
    """
    Genera una variable aleatoria
    con una distribución normal
    """
    random_number = get_random_number()
    square_root = math.sqrt(-2 * math.log1p(random_number - 1))
    sin = math.sin(2 * math.pi * random_number)
    value = square_root * sin
    if(value < 0):
        value = 0
    return value
def arrival(actual_simulated_time):
    """
    Recibe a un nuevo cliente.
    """
    global time
    global number_of_clients
    global number_of_arrivals
    global arrivals
    global lambda_parameter
    global max_service_time
    global main_times
    global clients, times
    
    
    time = actual_simulated_time
    
    clients.append(number_of_clients)
    times.append(time)
    
    number_of_clients += 1
    number_of_arrivals += 1
    arrivals.append(time)
    
    clients.append(number_of_clients)
    times.append(time)
    
    new_simulated_time = get_exponential(lambda_parameter)
    if(time + new_simulated_time < max_service_time):
        main_times['arrival_times'] = time + new_simulated_time
    if(number_of_clients == 1):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)
        

def server(actual_simulated_time):
    """
    Recibe como parametro el tiempo en el que se realizará
    la atención al cliente y posterior a ello lo atiende
    siempre y cuando hayan clientes por atender.
    """
    global time
    global number_of_clients
    global number_of_departures
    global departures
    global main_times
    global clients, times

    
    time = actual_simulated_time
    
    clients.append(number_of_clients)
    times.append(time)
    
    number_of_clients -= 1
    number_of_departures += 1
    departures.append(time)
    
    clients.append(number_of_clients)
    times.append(time)
    
    if(number_of_clients > 0):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)

M = 9999999999 #big big number 
lambda_parameter = 3
max_service_time = 10


promedio_desempeño_tiempo_fila_anterior = 0
promedio_desempeño_tiempo_fila = 0
varianza_tiempo_fila = 0

promedios = []
varianzas = []
tiempo_fila = []
for i in range(100):
    tiempo_fila.clear()
    #print("i= ",i)
    time = 0 
    number_of_clients = 0
    number_of_arrivals = 0
    number_of_departures = 0
    
    simulation_time_after_last_client = 0
    actual_simulated_time = 0
    
    mean_system_time = 0
    mean_queue_time = 0
    
    departures = []
    arrivals = []
    services_time = []
    
    clients = [0] # Eje Y
    times = [0]   # Eje X
    
    
    
    main_times = {
        'arrival_times': M,
        'departure_times': M
    }
    
    
    
    actual_simulated_time = get_exponential(lambda_parameter)
    
    if actual_simulated_time > max_service_time:
        simulation_time_after_last_client = 0
        mean_system_time = 0
        mean_queue_time = 0
    else:
        arrival(actual_simulated_time)
            
        while(main_times['arrival_times'] != M or main_times['departure_times'] != M):
            if(main_times['arrival_times'] < main_times['departure_times']):
                new_simulated_time = main_times['arrival_times']
                main_times['arrival_times'] = M
                arrival(new_simulated_time)
            if(main_times['departure_times'] < main_times['arrival_times']):
                new_simulated_time = main_times['departure_times']
                main_times['departure_times'] = M
                server(new_simulated_time)
                        
        simulation_time_after_last_client = max(0, time - max_service_time)
        accumulated_1 = 0
        accumulated_2 = 0
        counter = 0
        while counter < number_of_arrivals:
            accumulated_1 += departures[counter] - arrivals[counter]
            accumulated_2 += departures[counter] - arrivals[counter] - services_time[counter]
            counter += 1
                        
        mean_system_time = accumulated_1 / number_of_arrivals
        mean_queue_time = accumulated_2 / number_of_arrivals
                            
    #print("Tiempo de simulación: {}".format(time))
    #print("Tiempo de servicio despues del cierre: {}".format(simulation_time_after_last_client))
    #print("Tiempo medio del sistema: {}".format(mean_system_time))
    #print("Tiempo medio de la cola: {}".format(mean_queue_time))
    #print("\nMáximo número de clientes en el sistema: {}".format(max(clients)))
    
    
    if i >= 1:
        promedio_desempeño_tiempo_fila = media_recursiva(promedio_desempeño_tiempo_fila, mean_queue_time, i)
    else:
        promedio_desempeño_tiempo_fila = mean_queue_time
        
    if i >= 2:
        varianza_tiempo_fila = varianza_recursiva(
            promedio_desempeño_tiempo_fila_anterior,
            promedio_desempeño_tiempo_fila,
            varianza_tiempo_fila,
            i)
        
    promedio_desempeño_tiempo_fila_anterior = promedio_desempeño_tiempo_fila
    promedios.append(promedio_desempeño_tiempo_fila)
    varianzas.append(varianza_tiempo_fila)
    
print("Para k=100:")
print("La media de la muestra es de:",promedios[100-1])
print("La varianza de la muestra es de: ",varianzas[100-1],"\n")

Para k=100:
La media de la muestra es de: 3.5390362001989444
La varianza de la muestra es de:  3.7643584987254766 



## **k=1000**

In [2]:
def media_recursiva(media_anterior, valor_actual, j):
    x=(media_anterior + ( (valor_actual - media_anterior) / (j + 1.0)))
    return x

def varianza_recursiva(media_anterior, media_actual, varianza_anterior, j):
    p1 = (1 - ( 1.0 / j)) * varianza_anterior
    p2 = ((media_actual - media_anterior) ** 2 ) * (j + 1)
    s=(p1 + p2)
    return s

def get_random_number():
    """
    Genera una variable aleatoria
    con una distribución uniforme.
    """
    return random.random()

def get_exponential(lambda_parameter):
    """
    Genera una variable aleatoria
    con una distribución exponencial.
    """
    random_number = get_random_number()
    value = - (math.log(1 - random_number) / float(lambda_parameter))
    return value

def get_normal():
    """
    Genera una variable aleatoria
    con una distribución normal
    """
    random_number = get_random_number()
    square_root = math.sqrt(-2 * math.log1p(random_number - 1))
    sin = math.sin(2 * math.pi * random_number)
    value = square_root * sin
    if(value < 0):
        value = 0
    return value
def arrival(actual_simulated_time):
    """
    Recibe a un nuevo cliente.
    """
    global time
    global number_of_clients
    global number_of_arrivals
    global arrivals
    global lambda_parameter
    global max_service_time
    global main_times
    global clients, times
    
    
    time = actual_simulated_time
    
    clients.append(number_of_clients)
    times.append(time)
    
    number_of_clients += 1
    number_of_arrivals += 1
    arrivals.append(time)
    
    clients.append(number_of_clients)
    times.append(time)
    
    new_simulated_time = get_exponential(lambda_parameter)
    if(time + new_simulated_time < max_service_time):
        main_times['arrival_times'] = time + new_simulated_time
    if(number_of_clients == 1):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)
        

def server(actual_simulated_time):
    """
    Recibe como parametro el tiempo en el que se realizará
    la atención al cliente y posterior a ello lo atiende
    siempre y cuando hayan clientes por atender.
    """
    global time
    global number_of_clients
    global number_of_departures
    global departures
    global main_times
    global clients, times

    
    time = actual_simulated_time
    
    clients.append(number_of_clients)
    times.append(time)
    
    number_of_clients -= 1
    number_of_departures += 1
    departures.append(time)
    
    clients.append(number_of_clients)
    times.append(time)
    
    if(number_of_clients > 0):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)

M = 9999999999 #big big number 
lambda_parameter = 3
max_service_time = 10


promedio_desempeño_tiempo_fila_anterior = 0
promedio_desempeño_tiempo_fila = 0
varianza_tiempo_fila = 0

promedios = []
varianzas = []

for i in range(1000):

    #print("i= ",i)
    time = 0 
    number_of_clients = 0
    number_of_arrivals = 0
    number_of_departures = 0
    
    simulation_time_after_last_client = 0
    actual_simulated_time = 0
    
    mean_system_time = 0
    mean_queue_time = 0
    
    departures = []
    arrivals = []
    services_time = []
    
    clients = [0] # Eje Y
    times = [0]   # Eje X
    
    
    
    main_times = {
        'arrival_times': M,
        'departure_times': M
    }
    
    
    
    actual_simulated_time = get_exponential(lambda_parameter)
    
    if actual_simulated_time > max_service_time:
        simulation_time_after_last_client = 0
        mean_system_time = 0
        mean_queue_time = 0
    else:
        arrival(actual_simulated_time)
            
        while(main_times['arrival_times'] != M or main_times['departure_times'] != M):
            if(main_times['arrival_times'] < main_times['departure_times']):
                new_simulated_time = main_times['arrival_times']
                main_times['arrival_times'] = M
                arrival(new_simulated_time)
            if(main_times['departure_times'] < main_times['arrival_times']):
                new_simulated_time = main_times['departure_times']
                main_times['departure_times'] = M
                server(new_simulated_time)
                        
        simulation_time_after_last_client = max(0, time - max_service_time)
        accumulated_1 = 0
        accumulated_2 = 0
        counter = 0
        while counter < number_of_arrivals:
            accumulated_1 += departures[counter] - arrivals[counter]
            accumulated_2 += departures[counter] - arrivals[counter] - services_time[counter]
            counter += 1
                        
        mean_system_time = accumulated_1 / number_of_arrivals
        mean_queue_time = accumulated_2 / number_of_arrivals
                            
    #print("Tiempo de simulación: {}".format(time))
    #print("Tiempo de servicio despues del cierre: {}".format(simulation_time_after_last_client))
    #print("Tiempo medio del sistema: {}".format(mean_system_time))
    #print("Tiempo medio de la cola: {}".format(mean_queue_time))
    #print("\nMáximo número de clientes en el sistema: {}".format(max(clients)))
    if i >= 1:
        promedio_desempeño_tiempo_fila = media_recursiva(promedio_desempeño_tiempo_fila, mean_queue_time, i)
    else:
        promedio_desempeño_tiempo_fila = mean_queue_time
        
    if i >= 2:
        varianza_tiempo_fila = varianza_recursiva(
            promedio_desempeño_tiempo_fila_anterior,
            promedio_desempeño_tiempo_fila,
            varianza_tiempo_fila,
            i)
        
    promedio_desempeño_tiempo_fila_anterior = promedio_desempeño_tiempo_fila
    promedios.append(promedio_desempeño_tiempo_fila)
    varianzas.append(varianza_tiempo_fila)

print("Para k=1000:")
print("La media de la muestra es de:",promedios[1000-1])
print("La varianza de la muestra es de: ",varianzas[1000-1],"\n")

Para k=1000:
La media de la muestra es de: 3.7798652355701208
La varianza de la muestra es de:  4.334439975627615 



In [3]:
def media_recursiva(media_anterior, valor_actual, j):
    x=(media_anterior + ( (valor_actual - media_anterior) / (j + 1.0)))
    return x

def varianza_recursiva(media_anterior, media_actual, varianza_anterior, j):
    p1 = (1 - ( 1.0 / j)) * varianza_anterior
    p2 = ((media_actual - media_anterior) ** 2 ) * (j + 1)
    s=(p1 + p2)
    return s

def get_random_number():
    """
    Genera una variable aleatoria
    con una distribución uniforme.
    """
    return random.random()

def get_exponential(lambda_parameter):
    """
    Genera una variable aleatoria
    con una distribución exponencial.
    """
    random_number = get_random_number()
    value = - (math.log(1 - random_number) / float(lambda_parameter))
    return value

def get_normal():
    """
    Genera una variable aleatoria
    con una distribución normal
    """
    random_number = get_random_number()
    square_root = math.sqrt(-2 * math.log1p(random_number - 1))
    sin = math.sin(2 * math.pi * random_number)
    value = square_root * sin
    if(value < 0):
        value = 0
    return value
def arrival(actual_simulated_time):
    """
    Recibe a un nuevo cliente.
    """
    global time
    global number_of_clients
    global number_of_arrivals
    global arrivals
    global lambda_parameter
    global max_service_time
    global main_times
    global clients, times
    
    
    time = actual_simulated_time
    
    clients.append(number_of_clients)
    times.append(time)
    
    number_of_clients += 1
    number_of_arrivals += 1
    arrivals.append(time)
    
    clients.append(number_of_clients)
    times.append(time)
    
    new_simulated_time = get_exponential(lambda_parameter)
    if(time + new_simulated_time < max_service_time):
        main_times['arrival_times'] = time + new_simulated_time
    if(number_of_clients == 1):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)
        

def server(actual_simulated_time):
    """
    Recibe como parametro el tiempo en el que se realizará
    la atención al cliente y posterior a ello lo atiende
    siempre y cuando hayan clientes por atender.
    """
    global time
    global number_of_clients
    global number_of_departures
    global departures
    global main_times
    global clients, times

    
    time = actual_simulated_time
    
    clients.append(number_of_clients)
    times.append(time)
    
    number_of_clients -= 1
    number_of_departures += 1
    departures.append(time)
    
    clients.append(number_of_clients)
    times.append(time)
    
    if(number_of_clients > 0):
        simulated_service_time = get_normal()
        main_times['departure_times'] = time + simulated_service_time
        services_time.append(simulated_service_time)

M = 9999999999 #big big number 
lambda_parameter = 3
max_service_time = 10


promedio_desempeño_tiempo_fila_anterior = 0
promedio_desempeño_tiempo_fila = 0
varianza_tiempo_fila = 0

promedios = []
varianzas = []

for i in range(10000):

    #print("i= ",i)
    time = 0 
    number_of_clients = 0
    number_of_arrivals = 0
    number_of_departures = 0
    
    simulation_time_after_last_client = 0
    actual_simulated_time = 0
    
    mean_system_time = 0
    mean_queue_time = 0
    
    departures = []
    arrivals = []
    services_time = []
    
    clients = [0] # Eje Y
    times = [0]   # Eje X
    
    
    
    main_times = {
        'arrival_times': M,
        'departure_times': M
    }
    
    
    
    actual_simulated_time = get_exponential(lambda_parameter)
    
    if actual_simulated_time > max_service_time:
        simulation_time_after_last_client = 0
        mean_system_time = 0
        mean_queue_time = 0
    else:
        arrival(actual_simulated_time)
            
        while(main_times['arrival_times'] != M or main_times['departure_times'] != M):
            if(main_times['arrival_times'] < main_times['departure_times']):
                new_simulated_time = main_times['arrival_times']
                main_times['arrival_times'] = M
                arrival(new_simulated_time)
            if(main_times['departure_times'] < main_times['arrival_times']):
                new_simulated_time = main_times['departure_times']
                main_times['departure_times'] = M
                server(new_simulated_time)
                        
        simulation_time_after_last_client = max(0, time - max_service_time)
        accumulated_1 = 0
        accumulated_2 = 0
        counter = 0
        while counter < number_of_arrivals:
            accumulated_1 += departures[counter] - arrivals[counter]
            accumulated_2 += departures[counter] - arrivals[counter] - services_time[counter]
            counter += 1
                        
        mean_system_time = accumulated_1 / number_of_arrivals
        mean_queue_time = accumulated_2 / number_of_arrivals
                            
    #print("Tiempo de simulación: {}".format(time))
    #print("Tiempo de servicio despues del cierre: {}".format(simulation_time_after_last_client))
    #print("Tiempo medio del sistema: {}".format(mean_system_time))
    #print("Tiempo medio de la cola: {}".format(mean_queue_time))
    #print("\nMáximo número de clientes en el sistema: {}".format(max(clients)))
    if i >= 1:
        promedio_desempeño_tiempo_fila = media_recursiva(promedio_desempeño_tiempo_fila, mean_queue_time, i)
    else:
        promedio_desempeño_tiempo_fila = mean_queue_time
        
    if i >= 2:
        varianza_tiempo_fila = varianza_recursiva(
            promedio_desempeño_tiempo_fila_anterior,
            promedio_desempeño_tiempo_fila,
            varianza_tiempo_fila,
            i)
        
    promedio_desempeño_tiempo_fila_anterior = promedio_desempeño_tiempo_fila
    promedios.append(promedio_desempeño_tiempo_fila)
    varianzas.append(varianza_tiempo_fila)
    



print("Para k=10000:")
print("La media de la muestra es de:",promedios[10000-1])
print("La varianza de la muestra es de: ",varianzas[10000-1],"\n")

Para k=10000:
La media de la muestra es de: 3.8488919018375283
La varianza de la muestra es de:  4.675843407340376 



# **K=10000**